In [1]:
import datajoint as dj
import numpy as np
import json
from subprocess import call
import yaml
import sys, os
import pandas as pd
import ray

sys.path.append('./lib')
from utilities import *
sys.path.append('../lib')
from utils import run

In [14]:
#credFiles= '/data/Github/VaultBrain/credFiles_aws.yaml'
credFiles= '/Users/kuiqian/Github/VaultBrain/credFiles.yaml'
dj.config['database.host'] = get_dj_creds(credFiles)['database.host']
dj.config['database.user'] = get_dj_creds(credFiles)['database.user']
dj.config['database.port'] = get_dj_creds(credFiles)['database.port']
dj.config['database.password'] = get_dj_creds(credFiles)['database.password']
dj.conn()

./lib/utilities.py:14: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))


DataJoint connection (connected) kui@ucsd-demo-db.datajoint.io:3306

In [15]:
schema = dj.schema('kui_diffusionmap')
schema.spawn_missing_classes()

In [16]:
stack = 'MD594'
#yaml_file = 'shape_params-aws.yaml'
yaml_file = 'shape_params.yaml'
img_file = '/CSHL_hsv/'+stack+'/'
scripts_dir = os.environ['REPO_DIR']

In [5]:
Section.drop()

`kui_diffusionmap`.`section` (264 tuples)
`kui_diffusionmap`.`__image_map` (68 tuples)
Proceed? [yes, No]: yes


InternalError: (1051, "Unknown table 'kui_diffusionmap.__image_map'")

In [26]:
@schema
class Section(dj.Manual):
    definition = """
    section_id  : int   # section number
    """

In [7]:
import pickle
fname = os.path.join('CSHL_data_processed', stack, 'All_patch_locations.pkl')
all_patch_locations = pickle.load(open(os.environ['ROOT_DIR']+fname, 'rb'), encoding='latin1')

In [23]:
valid_sections = []
for section in all_patch_locations.keys():
    if all_patch_locations[section]:
        valid_sections.append(section)

In [30]:
for section in valid_sections:
    print("\nAdding "+str(section)+' to the database')
    Section.insert1(dict(section_id=section)
                      ,skip_duplicates=True)


Adding 94 to the database

Adding 95 to the database

Adding 96 to the database

Adding 97 to the database

Adding 98 to the database

Adding 99 to the database

Adding 100 to the database

Adding 101 to the database

Adding 102 to the database

Adding 103 to the database

Adding 104 to the database

Adding 105 to the database

Adding 106 to the database

Adding 107 to the database

Adding 108 to the database

Adding 109 to the database

Adding 110 to the database

Adding 111 to the database

Adding 112 to the database

Adding 113 to the database

Adding 114 to the database

Adding 115 to the database

Adding 116 to the database

Adding 117 to the database

Adding 118 to the database

Adding 119 to the database

Adding 120 to the database

Adding 121 to the database

Adding 122 to the database

Adding 123 to the database

Adding 124 to the database

Adding 125 to the database

Adding 126 to the database

Adding 127 to the database

Adding 128 to the database

Adding 129 to the databas

In [5]:
Section.fetch(as_dict=True)

[OrderedDict([('section_id', 94)]),
 OrderedDict([('section_id', 95)]),
 OrderedDict([('section_id', 96)]),
 OrderedDict([('section_id', 97)]),
 OrderedDict([('section_id', 98)]),
 OrderedDict([('section_id', 99)]),
 OrderedDict([('section_id', 100)]),
 OrderedDict([('section_id', 101)]),
 OrderedDict([('section_id', 102)]),
 OrderedDict([('section_id', 103)]),
 OrderedDict([('section_id', 104)]),
 OrderedDict([('section_id', 105)]),
 OrderedDict([('section_id', 106)]),
 OrderedDict([('section_id', 107)]),
 OrderedDict([('section_id', 108)]),
 OrderedDict([('section_id', 109)]),
 OrderedDict([('section_id', 110)]),
 OrderedDict([('section_id', 111)]),
 OrderedDict([('section_id', 112)]),
 OrderedDict([('section_id', 113)]),
 OrderedDict([('section_id', 114)]),
 OrderedDict([('section_id', 115)]),
 OrderedDict([('section_id', 116)]),
 OrderedDict([('section_id', 117)]),
 OrderedDict([('section_id', 118)]),
 OrderedDict([('section_id', 119)]),
 OrderedDict([('section_id', 120)]),
 Ordere

In [17]:
schema.jobs

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__image_map,3988c7f88ebcb58c6ce932b957b6f332,reserved,=BLOB=,,=BLOB=,kui@76.176.26.71,Kuis-MacBook-Pro.local,10064,19848,2019-07-10 15:16:38


In [6]:
(schema.jobs & 'status="reserved"' ).delete()

In [24]:
ImageMap.drop()

`kui_diffusionmap`.`__image_map` (1 tuples)
Proceed? [yes, No]: yes


KeyboardInterrupt: 

In [18]:
ImageMap.fetch(as_dict=True)

[OrderedDict([('section_id', 94), ('structure_number', 1)]),
 OrderedDict([('section_id', 95), ('structure_number', 1)]),
 OrderedDict([('section_id', 96), ('structure_number', 1)]),
 OrderedDict([('section_id', 97), ('structure_number', 1)]),
 OrderedDict([('section_id', 98), ('structure_number', 1)]),
 OrderedDict([('section_id', 99), ('structure_number', 1)]),
 OrderedDict([('section_id', 100), ('structure_number', 1)]),
 OrderedDict([('section_id', 101), ('structure_number', 1)]),
 OrderedDict([('section_id', 102), ('structure_number', 1)]),
 OrderedDict([('section_id', 103), ('structure_number', 1)]),
 OrderedDict([('section_id', 104), ('structure_number', 1)]),
 OrderedDict([('section_id', 105), ('structure_number', 3)]),
 OrderedDict([('section_id', 106), ('structure_number', 3)]),
 OrderedDict([('section_id', 107), ('structure_number', 3)]),
 OrderedDict([('section_id', 108), ('structure_number', 3)]),
 OrderedDict([('section_id', 109), ('structure_number', 3)]),
 OrderedDict([

In [9]:
! python HSV_datajoint_v2.py

./lib/utilities.py:14: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))
Connecting kui@ucsd-demo-db.datajoint.io:3306
ALREADY DOWNLOADED FILE
./lib/utilities.py:27: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))
populating for  94
run cmd= python3 /Users/kuiqian/Github/shapeology_code/scripts//HSV_v2.py MD594 94 shape_params.yaml
/Users/kuiqian/Github/venv/shapeology_venv/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED 

ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE
/Users/kuiqian/Github/shapeology_code/scripts/lib/utils.py:22: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.D=yaml.load(open(yamlFile,'r'))
run cmd= aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD594/MD594_prep2_lossless_gray/MD594-IHC17-2015.08.26-16.15.19_MD594_1_0049_prep2_lossless_gray.tif /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD594/MD594_prep2_lossless_gray/MD594-IHC17-2015.08.26-16.15.19_MD594_1_0049_prep2_lossless_gray.tif
download: s3://mousebrainatlas-data/CSHL_data_processed/MD594/MD594_prep2_lossless_gray/MD594-IHC17-2015.08.26-16.15.19_MD594_1_0049_prep2_lossless_gray.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD594/MD594_prep2_lossless_gray/MD594-IHC17-2015.08.26-16.15.19_MD594_1_0049_prep2_lossless_gray.tif
run c

run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/VCA/VCA_101.tif s3://mousebrainatlas-data/CSHL_hsv/MD594/VCA/VCA_101.tif
upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/VCA/VCA_101.tif to s3://mousebrainatlas-data/CSHL_hsv/MD594/VCA/VCA_101.tif
101 VCA 1 / 1
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_grid_features/MD594/101.pkl s3://mousebrainatlas-data/CSHL_grid_features/MD594/101.pkl
upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_grid_features/MD594/101.pkl to s3://mousebrainatlas-data/CSHL_grid_features/MD594/101.pkl
101 finished in 172.6 seconds
populating for  102
run cmd= python3 /Users/kuiqian/Github/shapeology_code/scripts//HSV_v2.py MD594 102 shape_params.yaml
/Users/kuiqian/Github/venv/shapeology_venv/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's docum

ALREADY DOWNLOADED FILE
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/VCA/VCA_105.tif s3://mousebrainatlas-data/CSHL_hsv/MD594/VCA/VCA_105.tif
upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/VCA/VCA_105.tif to s3://mousebrainatlas-data/CSHL_hsv/MD594/VCA/VCA_105.tif
105 VCA 1 / 3
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/DC/DC_105.tif s3://mousebrainatlas-data/CSHL_hsv/MD594/DC/DC_105.tif
upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/DC/DC_105.tif to s3://mousebrainatlas-data/CSHL_hsv/MD594/DC/DC_105.tif
105 DC 2 / 3
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/VCP/VCP_105.tif s3://mousebrainatlas-data/CSHL_hsv/MD594/VCP/VCP_105.tif
upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/VCP/VCP_105.tif to s3://mousebrainatlas-data/CSHL_hsv/MD594/VCP/VCP_105.tif
105 VCP 3 / 3
105 finished in 103.7 seconds
populating for  106
ru

run cmd= aws s3 cp s3://mousebrainatlas-data/CSHL_grid_features/MD594/108.pkl /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_grid_features/MD594/108.pkl
fatal error: An error occurred (404) when calling the HeadObject operation: Key "CSHL_grid_features/MD594/108.pkl" does not exist
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/VCP/VCP_108.tif s3://mousebrainatlas-data/CSHL_hsv/MD594/VCP/VCP_108.tif
upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/VCP/VCP_108.tif to s3://mousebrainatlas-data/CSHL_hsv/MD594/VCP/VCP_108.tif
108 VCP 1 / 3
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/VCA/VCA_108.tif s3://mousebrainatlas-data/CSHL_hsv/MD594/VCA/VCA_108.tif
upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/VCA/VCA_108.tif to s3://mousebrainatlas-data/CSHL_hsv/MD594/VCA/VCA_108.tif
108 VCA 2 / 3
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/DC/DC_108

...

download: s3://mousebrainatlas-data/CSHL_data_processed/MD594/MD594_prep2_lossless_gray/MD594-IHC24-2015.08.26-16.39.39_MD594_1_0070_prep2_lossless_gray.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD594/MD594_prep2_lossless_gray/MD594-IHC24-2015.08.26-16.39.39_MD594_1_0070_prep2_lossless_gray.tif
run cmd= aws s3 cp s3://mousebrainatlas-data/CSHL_grid_features/MD594/128.pkl /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_grid_features/MD594/128.pkl
fatal error: An error occurred (404) when calling the HeadObject operation: Key "CSHL_grid_features/MD594/128.pkl" does not exist
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/PBG/PBG_128.tif s3://mousebrainatlas-data/CSHL_hsv/MD594/PBG/PBG_128.tif
upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/PBG/PBG_128.tif to s3://mousebrainatlas-data/CSHL_hsv/MD594/PBG/PBG_128.tif
128 PBG 1 / 8
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/M

upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/SNR/SNR_130.tif to s3://mousebrainatlas-data/CSHL_hsv/MD594/SNR/SNR_130.tif
130 SNR 1 / 7
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/DC/DC_130.tif s3://mousebrainatlas-data/CSHL_hsv/MD594/DC/DC_130.tif
upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/DC/DC_130.tif to s3://mousebrainatlas-data/CSHL_hsv/MD594/DC/DC_130.tif
130 DC 2 / 7
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/Sp5I/Sp5I_130.tif s3://mousebrainatlas-data/CSHL_hsv/MD594/Sp5I/Sp5I_130.tif
upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/Sp5I/Sp5I_130.tif to s3://mousebrainatlas-data/CSHL_hsv/MD594/Sp5I/Sp5I_130.tif
130 Sp5I 3 / 7
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/VLL/VLL_130.tif s3://mousebrainatlas-data/CSHL_hsv/MD594/VLL/VLL_130.tif
upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/

133 VLL 2 / 7
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/Sp5I/Sp5I_133.tif s3://mousebrainatlas-data/CSHL_hsv/MD594/Sp5I/Sp5I_133.tif
upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/Sp5I/Sp5I_133.tif to s3://mousebrainatlas-data/CSHL_hsv/MD594/Sp5I/Sp5I_133.tif
133 Sp5I 3 / 7
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/DC/DC_133.tif s3://mousebrainatlas-data/CSHL_hsv/MD594/DC/DC_133.tif
upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/DC/DC_133.tif to s3://mousebrainatlas-data/CSHL_hsv/MD594/DC/DC_133.tif
133 DC 4 / 7
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/PBG/PBG_133.tif s3://mousebrainatlas-data/CSHL_hsv/MD594/PBG/PBG_133.tif
upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/PBG/PBG_133.tif to s3://mousebrainatlas-data/CSHL_hsv/MD594/PBG/PBG_133.tif
133 PBG 5 / 7
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBacku

upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/VLL/VLL_135.tif to s3://mousebrainatlas-data/CSHL_hsv/MD594/VLL/VLL_135.tif
135 VLL 3 / 10
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/Sp5I/Sp5I_135.tif s3://mousebrainatlas-data/CSHL_hsv/MD594/Sp5I/Sp5I_135.tif
upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/Sp5I/Sp5I_135.tif to s3://mousebrainatlas-data/CSHL_hsv/MD594/Sp5I/Sp5I_135.tif
135 Sp5I 4 / 10
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/SNR/SNR_135.tif s3://mousebrainatlas-data/CSHL_hsv/MD594/SNR/SNR_135.tif
upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/SNR/SNR_135.tif to s3://mousebrainatlas-data/CSHL_hsv/MD594/SNR/SNR_135.tif
135 SNR 5 / 10
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/Sp5O/Sp5O_135.tif s3://mousebrainatlas-data/CSHL_hsv/MD594/Sp5O/Sp5O_135.tif
upload: ../../../../BstemAtlasDataBackup/ucsd_brain

137 VLL 2 / 9
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/Sp5C/Sp5C_137.tif s3://mousebrainatlas-data/CSHL_hsv/MD594/Sp5C/Sp5C_137.tif
upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/Sp5C/Sp5C_137.tif to s3://mousebrainatlas-data/CSHL_hsv/MD594/Sp5C/Sp5C_137.tif
137 Sp5C 3 / 9
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/DC/DC_137.tif s3://mousebrainatlas-data/CSHL_hsv/MD594/DC/DC_137.tif
upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/DC/DC_137.tif to s3://mousebrainatlas-data/CSHL_hsv/MD594/DC/DC_137.tif
137 DC 4 / 9
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/SNR/SNR_137.tif s3://mousebrainatlas-data/CSHL_hsv/MD594/SNR/SNR_137.tif
upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/SNR/SNR_137.tif to s3://mousebrainatlas-data/CSHL_hsv/MD594/SNR/SNR_137.tif
137 SNR 5 / 9
^C
